In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.append('../../../pycore/')
from simulate import RBA_result
from utils import extract_details_from_rxnid

import matplotlib.pyplot as plt
import matplotlib.patches as mp
import seaborn as sns
import matplotlib.lines as mlines
%matplotlib inline

In [2]:
path_enz_mw = '../../../input/enz_mw_g_per_mmol.txt'
path_pro_mw = '../../../input/pro_mw_g_per_mmol.txt'

with open(path_enz_mw) as f:
    text = f.read().split('\n')
text = [i for i in text if i != '']
enz_mw = dict()
for i in text:
    k,v = i.split('\t')
    enz_mw[k] = float(v)
    
with open(path_pro_mw) as f:
    text = f.read().split('\n')
text = [i for i in text if i != '']
pro_mw = dict()
for i in text:
    k,v = i.split('\t')
    pro_mw[k] = float(v)

In [3]:
with open('../../../input/RBA_kapp_batch_per_hour.txt') as f:
    x = f.read().split('\n')
x = [i for i in x if i not in ['', '/']]

kapps = dict()
for line in x:
    rxn,kapp = line.split(' ')
    kapps[rxn[1:-1]] = float(kapp)

In [4]:
flist = ['vglc_0_5', 'vglc_1_0', 'vglc_2_0', 'vglc_3_0', 'vglc_4_0', 'vglc_5_0',
         'vglc_5_5', 'vglc_5_6', 'vglc_5_7', 'vglc_5_8', 'vglc_5_9', 'vglc_6_0', 'vglc_7_0',
         'vglc_8_0', 'vglc_9_0', 'vglc_10_0', 'vglc_11_0', 'vglc_12_0',
         'vglc_13_0', 'vglc_14_0', 'vglc_15_0', 'vglc_16_0', 'vglc_17_0']
reslist = []
for fdir in flist:
    res = RBA_result(biom_id=None)
    res.load_from_json('./results/' + fdir + '/RBA_result.json')
    reslist.append(res)

In [5]:
rxns_check = ['ENZLOAD-DBTS_c_FWD-YNR057C', 'ENZLOAD-DBTS_c_REV-YNR057C',
              'ENZLOAD-CYCCL_m_FWD-YAL039C', 'ENZLOAD-CYCCL_m_REV-YAL039C',
              'ENZLOAD-DADK_c_FWD-YDL166C', 'ENZLOAD-DADK_c_REV-YDL166C',
              'ENZLOAD-BTDDH_c_FWD-YAL060W', 'ENZLOAD-BTDDH_c_REV-YAL060W']
spec_pairs = [('ENZLOAD-ACACT40ir_c_REV-YPL028W', 'ENZLOAD-ACACT40ir_c_FWD-YPL028W'),
              ('ENZLOAD-ACACT40ir_m_REV-YPL028W', 'ENZLOAD-ACACT40ir_m_FWD-YPL028W'),
              ('ENZLOAD-TPI_c_REV-YDR050C', 'ENZLOAD-TPI_c_FWD-YDR050C')]

In [6]:
N = len(flist)
for i in range(0,N):
    fname = flist[i]
    res = reslist[i]
    gr = res.growth_rate
    if 'EX_etoh_e' in res.metabolic_flux.keys():
        etoh = res.metabolic_flux['EX_etoh_e']
    else:
        etoh = 0
    print(fname, ', GR =', round(gr,4), ', ETOH =', round(etoh,4))
    enzloads = {k:v for k,v in res.raw_flux.items() \
                if k.split('-')[0] == 'ENZLOAD' and v > 0}
    eload_raw = res.proteome_capacity_usage
    
    load_calc_denominator = 0.55 * res.raw_flux['BIOSYN-PROTTOBIO']
    
    eload_excess = 0
    for spec_rev,spec_fwd in spec_pairs:
        if spec_rev in enzloads.keys():
            # Subtract exchange reverse load
            eload = res.raw_flux[spec_rev] * enz_mw[spec_rev]
            print(spec_rev, round(eload / load_calc_denominator,4))
            eload_excess += eload
            
            # Subtract exchange forward load
            eload = res.raw_flux[spec_rev] * enz_mw[spec_fwd] * \
                kapps['RXN-'+spec_rev[8:]] / kapps['RXN-'+spec_fwd[8:]]
            eload_excess += eload
            print(spec_fwd, round(eload / load_calc_denominator,4))
        
    for rxn in rxns_check:
        if rxn in enzloads.keys():
            eload = res.raw_flux[rxn] * enz_mw[rxn]
            print(rxn, round(eload / load_calc_denominator,4))
            eload_excess += eload
            
    eload_new = eload_raw - eload_excess / load_calc_denominator
    print('Pre-corrected capacity usage:', round(eload_raw, 4))
    print('Corrected capacity usage:', round(eload_new,4))

vglc_0_5 , GR = 0.0411 , ETOH = 0
ENZLOAD-ACACT40ir_m_REV-YPL028W 0.2917
ENZLOAD-ACACT40ir_m_FWD-YPL028W 0.6385
Pre-corrected capacity usage: 1.0
Corrected capacity usage: 0.0697
vglc_1_0 , GR = 0.0853 , ETOH = 0
ENZLOAD-ACACT40ir_m_REV-YPL028W 0.2691
ENZLOAD-ACACT40ir_m_FWD-YPL028W 0.5891
Pre-corrected capacity usage: 1.0
Corrected capacity usage: 0.1418
vglc_2_0 , GR = 0.1723 , ETOH = 0
ENZLOAD-ACACT40ir_m_REV-YPL028W 0.2259
ENZLOAD-ACACT40ir_m_FWD-YPL028W 0.4943
Pre-corrected capacity usage: 1.0
Corrected capacity usage: 0.2798
vglc_3_0 , GR = 0.2577 , ETOH = 0
ENZLOAD-ACACT40ir_m_REV-YPL028W 0.1839
ENZLOAD-ACACT40ir_m_FWD-YPL028W 0.4026
Pre-corrected capacity usage: 1.0
Corrected capacity usage: 0.4134
vglc_4_0 , GR = 0.3415 , ETOH = 0
ENZLOAD-ACACT40ir_m_REV-YPL028W 0.1433
ENZLOAD-ACACT40ir_m_FWD-YPL028W 0.3137
Pre-corrected capacity usage: 1.0
Corrected capacity usage: 0.543
vglc_5_0 , GR = 0.4238 , ETOH = 0
ENZLOAD-ACACT40ir_m_REV-YPL028W 0.1039
ENZLOAD-ACACT40ir_m_FWD-YPL028W 0

In [4]:
flist = ['vglc_12_5246']
reslist = []
for fdir in flist:
    res = RBA_result(biom_id=None)
    res.load_from_json('./results/' + fdir + '/RBA_result.json')
    reslist.append(res)

In [5]:
rxns_check = ['ENZLOAD-DBTS_c_FWD-YNR057C', 'ENZLOAD-DBTS_c_REV-YNR057C',
              'ENZLOAD-CYCCL_m_FWD-YAL039C', 'ENZLOAD-CYCCL_m_REV-YAL039C',
              'ENZLOAD-DADK_c_FWD-YDL166C', 'ENZLOAD-DADK_c_REV-YDL166C',
              'ENZLOAD-BTDDH_c_FWD-YAL060W', 'ENZLOAD-BTDDH_c_REV-YAL060W']
spec_pairs = [('ENZLOAD-ACACT40ir_c_REV-YPL028W', 'ENZLOAD-ACACT40ir_c_FWD-YPL028W'),
              ('ENZLOAD-ACACT40ir_m_REV-YPL028W', 'ENZLOAD-ACACT40ir_m_FWD-YPL028W'),
              ('ENZLOAD-TPI_c_REV-YDR050C', 'ENZLOAD-TPI_c_FWD-YDR050C')]

In [6]:
N = len(flist)
for i in range(0,N):
    fname = flist[i]
    res = reslist[i]
    gr = res.growth_rate
    if 'EX_etoh_e' in res.metabolic_flux.keys():
        etoh = res.metabolic_flux['EX_etoh_e']
    else:
        etoh = 0
    print(fname, ', GR =', round(gr,4), ', ETOH =', round(etoh,4))
    enzloads = {k:v for k,v in res.raw_flux.items() \
                if k.split('-')[0] == 'ENZLOAD' and v > 0}
    eload_raw = res.proteome_capacity_usage
    
    load_calc_denominator = 0.55 * res.raw_flux['BIOSYN-PROTTOBIO']
    
    eload_excess = 0
    for spec_rev,spec_fwd in spec_pairs:
        if spec_rev in enzloads.keys():
            # Subtract exchange reverse load
            eload = res.raw_flux[spec_rev] * enz_mw[spec_rev]
            print(spec_rev, round(eload / load_calc_denominator,4))
            eload_excess += eload
            
            # Subtract exchange forward load
            eload = res.raw_flux[spec_rev] * enz_mw[spec_fwd] * \
                kapps['RXN-'+spec_rev[8:]] / kapps['RXN-'+spec_fwd[8:]]
            eload_excess += eload
            print(spec_fwd, round(eload / load_calc_denominator,4))
        
    for rxn in rxns_check:
        if rxn in enzloads.keys():
            eload = res.raw_flux[rxn] * enz_mw[rxn]
            print(rxn, round(eload / load_calc_denominator,4))
            eload_excess += eload
            
    eload_new = eload_raw - eload_excess / load_calc_denominator
    print('Pre-corrected capacity usage:', round(eload_raw, 4))
    print('Corrected capacity usage:', round(eload_new,4))

vglc_12_5246 , GR = 0.4623 , ETOH = 15.196
ENZLOAD-DBTS_c_FWD-YNR057C 0.095
ENZLOAD-DBTS_c_REV-YNR057C 0.095
Pre-corrected capacity usage: 1.0
Corrected capacity usage: 0.8101


In [16]:
rxns_check = ['ENZLOAD-DBTS_c_FWD-YNR057C', 'ENZLOAD-DBTS_c_REV-YNR057C',
              'ENZLOAD-CYCCL_m_FWD-YAL039C', 'ENZLOAD-CYCCL_m_REV-YAL039C',
              'ENZLOAD-DADK_c_FWD-YDL166C', 'ENZLOAD-DADK_c_REV-YDL166C',
              'ENZLOAD-BTDDH_c_FWD-YAL060W', 'ENZLOAD-BTDDH_c_REV-YAL060W',
              'ENZLOAD-FUM_m_FWD-YPL262W_m', 'ENZLOAD-FUM_m_REV-YPL262W_m']
spec_rev = 'ENZLOAD-ACACT40ir_m_REV-YPL028W';
spec_fwd = 'ENZLOAD-ACACT40ir_m_FWD-YPL028W'
spec_rev2 = 'ENZLOAD-ACACT40ir_c_REV-YPL028W'
spec_fwd2 = 'ENZLOAD-ACACT40ir_c_FWD-YPL028W'

In [17]:
i = 3
fname = flist[i]
print(fname)
res = reslist[i]
gr = res.growth_rate    
enzloads = {k:v for k,v in res.raw_flux.items() \
            if k.split('-')[0] == 'ENZLOAD' and v > 0}
dummysum = res.raw_flux['BIOSYN-PROTDUMMY']
if 'BIOSYN-PROTDUMMY2' in res.raw_flux.keys():
    dummy_modeled_load = res.raw_flux['BIOSYN-PROTDUMMY2']
else:
    dummy_modeled_load = 0
pload = res.raw_flux['BIOSYN-PROTMODELED']
procap = res.raw_flux['BIOSYN-PROTTOBIO']
eload_raw = (pload - dummy_modeled_load) / (procap - (dummysum - dummy_modeled_load))

load_calc_denominator = 0.55 * res.raw_flux['BIOSYN-PROTTOBIO']

eload_excess = 0
if spec_rev in enzloads.keys():
    eload = res.raw_flux[spec_rev] * enz_mw[spec_rev]
    print(spec_rev, round(eload / load_calc_denominator,4))
    eload_excess += eload

    eload = res.raw_flux[spec_rev] * enz_mw[spec_fwd] * kapps['RXN-'+spec_rev[8:]] / kapps['RXN-'+spec_fwd[8:]]
    eload_excess += eload
    print(spec_fwd, round(eload / load_calc_denominator,4))

if spec_rev2 in enzloads.keys():
    eload = res.raw_flux[spec_rev2] * enz_mw[spec_rev2]
    print(spec_rev2, round(eload / load_calc_denominator,4))
    eload_excess += eload

    eload = res.raw_flux[spec_rev2] * enz_mw[spec_fwd2] * kapps['RXN-'+spec_rev2[8:]] / kapps['RXN-'+spec_fwd2[8:]]
    eload_excess += eload
    print(spec_fwd2, round(eload / load_calc_denominator,4))

for rxn in rxns_check:
    if rxn in enzloads.keys():
        eload = res.raw_flux[rxn] * enz_mw[rxn]
        print(rxn, round(eload / load_calc_denominator,4))
        eload_excess += eload

eload_new = eload_raw - eload_excess / load_calc_denominator
print('Pre-corrected capacity usage:', round(eload_raw, 4))
print('Corrected capacity usage:', round(eload_new + 1e-4,4))

vglc_3_0
ENZLOAD-ACACT40ir_m_REV-YPL028W 0.1879
ENZLOAD-ACACT40ir_m_FWD-YPL028W 0.6173
ENZLOAD-FUM_m_FWD-YPL262W_m 0.0
ENZLOAD-FUM_m_REV-YPL262W_m 0.0001
Pre-corrected capacity usage: 1.0
Corrected capacity usage: 0.1949


In [19]:
flist = ['vglc_5_0', 'vglc_5_1', 'vglc_5_2', 'vglc_5_3', 'vglc_5_4',
         'vglc_5_5', 'vglc_5_6', 'vglc_5_7', 'vglc_5_8', 'vglc_5_9',
         'vglc_6_0', 'vglc_7_0']
reslist = []
for fdir in flist:
    res = RBA_result(biom_id=None)
    res.load_from_json('./results/' + fdir + '/RBA_result.json')
    reslist.append(res)
    
N = len(flist)
for i in range(0,N):
    res = reslist[i]
    if 'EX_etoh_e' in res.metabolic_flux.keys():
        etoh = res.metabolic_flux['EX_etoh_e']
    else:
        etoh = 0
    print(flist[i], res.growth_rate, etoh)

vglc_5_0 0.437149047851563 0
vglc_5_1 0.445610046386719 0
vglc_5_2 0.454059536132813 0
vglc_5_3 0.462364196777344 0
vglc_5_4 0.469764709472656 0
vglc_5_5 0.471058135375977 0
vglc_5_6 0.471117119140625 0
vglc_5_7 0.471176435546875 0
vglc_5_8 0.471198033447266 0
vglc_5_9 0.471198172149658 0
vglc_6_0 0.47119140625 0
vglc_7_0 0.471168518066406 0
